In [23]:
import pandas as pd

import googlemaps
def google_distance():
    data=pd.read_csv('one_route_example.csv')
    
    API_key = ''#enter Google Maps API key
    gmaps = googlemaps.Client(key=API_key)
    distance=[[0 for i in range(len(data))]  for i in range(len(data))]
    for (i1, row1) in data.iterrows():
        LatOrigin = row1['latitude']
        LongOrigin = row1['longitude']
        origin = (LatOrigin, LongOrigin)
        origin_id = row1['route_id'] 
        for (i2, row2) in  data.iterrows():
            LatDestination = row2['latitude']
            LongDestination = row2['longitude']
            destination_id = row2['route_id']
            destination = (LatDestination, LongDestination)
            result = gmaps.distance_matrix(origin, destination, mode = 'driving')
            result_distance = result["rows"][0]["elements"][0]["distance"]["value"]
            result_time = result["rows"][0]["elements"][0]["duration"]["value"]
            distance[i1][i2]=float(result_time)
        
    return distance

In [24]:
import numpy as np
from scipy.spatial.distance import cdist
import pandas as pd

# Constants
POPULATION_SIZE = 50
CROSSOVER_RATE = 0.8
MUTATION_RATE = 0.02
GENERATIONS = 100


In [25]:
#adding warehouse information
distance_matrixN =np.array(google_distance())
distance_matrix=np.zeros([len(distance_matrixN)-1,len(distance_matrixN)-1])
for i in range(1,len(distance_matrixN)):
    for j in range(1,len(distance_matrixN)):
            distance_matrix[i-1,j-1]=distance_matrixN [i][j]

In [26]:
# Function to calculate the total distance of a route
def calculate_total_distance(route):
    total_distance = 0
    for i in range(len(route) - 1):
        total_distance += distance_matrix[route[i], route[i + 1]]
    total_distance+=distance_matrixN[0][route[0]+1]+distance_matrixN[0][route[-1]+1]
    return total_distance

In [27]:

# Function to create an initial population
def initialize_population(num_customers):
    return [np.random.permutation(num_customers) for _ in range(POPULATION_SIZE)]

In [28]:
# Function to perform tournament selection
def tournament_selection(population, fitness_values):
    selected_indices = np.random.choice(len(population), size=2, replace=False)
    return population[selected_indices[np.argmin(fitness_values[selected_indices])]]

In [29]:

# Function to perform order crossover (OX)
def order_crossover(parent1, parent2):
    start = np.random.randint(1, len(parent1) - 1)  # Exclude depot
    end = np.random.randint(start + 1, len(parent1))
    
    child = [-1] * len(parent1)
    child[start:end] = parent1[start:end]
    
    remaining_indices = [i for i in parent2 if i not in child]
    child[:start] = remaining_indices[:start]
    child[end:] = remaining_indices[start:]
    
    return child

In [30]:
# Function to perform mutation (swap)
def swap_mutation(route):
    mutated_route = route.copy()
    index1, index2 = np.random.choice(len(route) - 1, size=2, replace=False) + 1  # Exclude depot
    mutated_route[index1], mutated_route[index2] = mutated_route[index2], mutated_route[index1]
    return mutated_route

In [31]:
# Function to perform routing using genetic algorithm
def genetic_algorithm():
    num_customers = len(distance_matrixN)-1
    population = initialize_population(num_customers)
    
    for generation in range(GENERATIONS):
        fitness_values = np.array([calculate_total_distance([0] + route + [0]) for route in population])
        sorted_indices = np.argsort(fitness_values)
        
        # Elitism: Preserve the best individuals
        elite_size = int(0.1 * POPULATION_SIZE)
        elite = [population[i] for i in sorted_indices[:elite_size]]
        
        new_population = elite
        
        while len(new_population) < POPULATION_SIZE:
            if np.random.rand() < CROSSOVER_RATE:
                parent1 = tournament_selection(population, fitness_values)
                parent2 = tournament_selection(population, fitness_values)
                child = order_crossover(parent1, parent2)
            else:
                child = tournament_selection(population, fitness_values)
            
            if np.random.rand() < MUTATION_RATE:
                child = swap_mutation(child)
            
            new_population.append(child)
        
        population = np.array(new_population)
    
    # Find the best route and print the result
    best_route = population[np.argmin(fitness_values)]
    best_distance = calculate_total_distance([0] + best_route + [0])
    print(f"Best route: {[0] + best_route + [0]}")  # Add depot at the beginning and end
    print(f"Total distance: {best_distance}")
    return best_route

In [32]:
# Run the genetic algorithm
best=genetic_algorithm()

Best route: [2 0 1 3]
Total distance: 2423.0


In [33]:
import gmplot
import pandas as pd
apikey = '' # (your API key here)

data=pd.read_csv('one_route_example.csv')
a=data.iloc[0,3]
gmap = gmplot.GoogleMapPlotter(data.iloc[0,3], data.iloc[0,4], zoom = 14, apikey=apikey)

gmap.directions(
    (data.iloc[0,3],data.iloc[0,4]),
    (data.iloc[best[0]+1,3],data.iloc[best[0]+1,4]),
    title='Adsa')
for i in range(len(best)-1):
    gmap.directions(
    (data.iloc[best[i]+1,3],data.iloc[best[i]+1,4]),
    (data.iloc[best[i+1]+1,3],data.iloc[best[i+1]+1,4]))
gmap.directions(
    
    (data.iloc[best[i+1]+1,3],data.iloc[best[i+1]+1,4]),(data.iloc[0,3],data.iloc[0,4]))
gmap.draw('route.html')